In [2]:
import pandas as pd 
import numpy as np
import sqlite3 as sql
import os

# Instalar gdown para acceder al archivo en Drive\n"
%pip install gdown
import gdown


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ID del archivo de Google Drive
file_id = "1YEdQA-dDrK9o6_H7sIy3LqbDk-RnWf0d"
url = f"https://drive.google.com/uc?id={file_id}"

# Descargar el archivo
output = "dataset.csv"
gdown.download(url, output, quiet=False)

# Leer el archivo
try:
    df = pd.read_csv(output, encoding="ISO-8859-1")
    print("✅ Archivo cargado exitosamente")
    print(df.head())
    print(f"\nForma del dataset: {df.shape}")
except Exception as e:
    print(f"Error: {e}")

Downloading...
From: https://drive.google.com/uc?id=1YEdQA-dDrK9o6_H7sIy3LqbDk-RnWf0d
To: /workspaces/Big_Data/src/dataset.csv
100%|██████████| 12.4M/12.4M [00:00<00:00, 100MB/s] 


✅ Archivo cargado exitosamente
   OBJECTID                                     Shape radicado  \
0         1         (4713547.5105, 2247612.977400001)  1580885   
1         2         (4715357.8105, 2243807.580100002)  1585081   
2         3         (4710262.4967, 2249847.019700002)  1581508   
3         4  (4715512.4542000005, 2246920.8745000027)  1581862   
4         5   (4713597.201000002, 2252326.8505000006)  1578199   

                 fecha      hora  dia  periodo           clase  \
0  2017-05-05 00:00:00  02:00 PM    5     2017          Choque   
1  2017-06-06 00:00:00  11:20 AM    6     2017          Choque   
2  2017-05-10 00:00:00  12:00 PM   10     2017          Choque   
3  2017-05-12 00:00:00  05:30 PM   12     2017  Caida Ocupante   
4  2017-04-14 00:00:00  04:30 AM   14     2017          Choque   

           direccion                direccion_enc  ...           barrio  \
0  CL 32 Norte CR 69     CL  032   069  000 00000  ...          Rosales   
1   CL 4 Sur CR 43 B  CL 

# Problemática y dataset

1. Problema:  Identificar patrones y tendencias en los incidentes de tránsito, incluyendo su distribución geográfica y temporal.
2. Variables relevantes: 

*Categóricos: clase, dia_nombre, barrio, comuna, diseno.
*Numéricos: Coordenadas (longitud, latitud).
*Fecha y hora: fecha, hora.

3. Enlace: [Dataset](https://www.medellin.gov.co/geomedellin/datosAbiertos/275)

Se eligio este dataset con el fin de identificar puntos criticos de mayor indice de accidentabilidad en la ciudad de medellín, el fin es poder generar alertas tempranas en dichos puntos y realizar campañas de sensibilización a los ciudadanos y también implementar medidas como el uso de resaltos, implementación de señalizaciones y también instalación de semaforización.


# Modelo entidad–relación (ERD)

![Modelo_ER](docs/Modelo_ER.png)

El diagrama representa un sistema que organiza los incidentes de tránsito en tres niveles:


1. Comuna: Es la unidad territorial más amplia. Cada comuna agrupa varios barrios.

2. Barrio: Cada barrio pertenece a una sola comuna y actúa como punto intermedio entre la comuna y los incidentes.

3. Incidente: Contiene toda la información detallada de cada evento de tránsito: fecha, hora, gravedad, tipo de incidente, ubicación y coordenadas. Cada incidente ocurre en un único barrio.


En conjunto, el modelo muestra que:

°Una comuna puede tener muchos barrios.

°Un barrio puede tener muchos incidentes.

°Cada incidente queda conectado geográfica y administrativamente mediante su barrio.



# 3️⃣ Crea la base de datos e inserta información

In [ ]:
# Intentar crear la DB en el directorio actual; si falla por I/O, usar /tmp
db_file = 'incidentes_transito.db'
csv_file = 'dataset.csv'
db_path = db_file  # Ruta inicial

# Verificar si el directorio actual tiene permisos de escritura
if not os.access('.', os.W_OK):
    print("El directorio actual no tiene permisos de escritura. Usando directorio temporal...")
    temp_dir = tempfile.gettempdir()
    db_path = os.path.join(temp_dir, db_file)
    print(f"Usando ruta alternativa: {db_path}")
    
# Eliminar DB existente si hay
if os.path.exists(db_path):
    os.remove(db_path)
    print("Base de datos existente eliminada. Recreando...")
# Leer CSV
if not os.path.exists(csv_file):
    raise FileNotFoundError(f"El archivo {csv_file} no se encuentra en el directorio actual.")
df = pd.read_csv(csv_file, sep=',', encoding='latin1')
print(f"CSV cargado con {len(df)} filas.")
print(f"Columnas del DataFrame: {list(df.columns)}")
# Conectar a DB
try:
    conn = sql.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON;")
    print("Conexión a DB exitosa.")
except Exception as e:
    print(f"Error al conectar a DB: {e}")
    raise


CSV cargado con 46210 filas.
Columnas del DataFrame: ['OBJECTID', 'Shape', 'radicado', 'fecha', 'hora', 'dia', 'periodo', 'clase', 'direccion', 'direccion_enc', 'cbml', 'tipo_geocod', 'gravedad', 'barrio', 'comuna', 'diseno', 'dia_nombre', 'mes', 'mes_nombre', 'longitud', 'latitud', 'x_origen_nacional', 'y_origen_nacional']
Conexión a DB exitosa.


In [54]:
# Crear tablas
try:
    cursor.execute('''
    CREATE TABLE Ubicacion (
        cbml TEXT PRIMARY KEY,
        direccion TEXT,
        direccion_enc TEXT,
        barrio TEXT,
        comuna TEXT,
        longitud REAL,
        latitud REAL,
        x_origen_nacional REAL,
        y_origen_nacional REAL,
        Shape TEXT
    );
    ''')
    print("Tabla Ubicacion creada.")
    cursor.execute('''
    CREATE TABLE Tiempo (
        fecha TEXT,
        hora TEXT,
        dia INTEGER,
        periodo INTEGER,
        dia_nombre TEXT,
        mes INTEGER,
        mes_nombre TEXT,
        PRIMARY KEY (fecha, hora)
    );
    ''')
    print("Tabla Tiempo creada.")
    
    cursor.execute('''
    CREATE TABLE Tipo_Incidente (
        clase TEXT PRIMARY KEY,
        gravedad TEXT
    );
    ''')
    print("Tabla Tipo_Incidente creada.")
    cursor.execute('''
    CREATE TABLE Incidente (
        radicado TEXT PRIMARY KEY,
        clase TEXT,
        gravedad TEXT,
        diseno TEXT,
        tipo_geocod TEXT,
        OBJECTID INTEGER,
        cbml TEXT,
        fecha TEXT,
        hora TEXT,
        FOREIGN KEY (cbml) REFERENCES Ubicacion(cbml),
        FOREIGN KEY (fecha, hora) REFERENCES Tiempo(fecha, hora),
        FOREIGN KEY (clase) REFERENCES Tipo_Incidente(clase)
    );
    ''')
    print("Tabla Incidente creada.")
# Cargar datos
    ubicacion_data = df[['cbml', 'direccion', 'direccion_enc', 'barrio', 'comuna', 'longitud', 'latitud', 'x_origen_nacional', 'y_origen_nacional', 'Shape']].drop_duplicates(subset=['cbml'])
    ubicacion_data.to_sql('Ubicacion', conn, if_exists='append', index=False)
    print(f"Ubicacion cargada con {len(ubicacion_data)} filas.")
    
    tiempo_data = df[['fecha', 'hora', 'dia', 'periodo', 'dia_nombre', 'mes', 'mes_nombre']].drop_duplicates(subset=['fecha', 'hora'])
    tiempo_data.to_sql('Tiempo', conn, if_exists='append', index=False)
    print(f"Tiempo cargada con {len(tiempo_data)} filas.")
    
    tipo_incidente_data = df[['clase', 'gravedad']].drop_duplicates(subset=['clase'])
    tipo_incidente_data.to_sql('Tipo_Incidente', conn, if_exists='append', index=False)
    print(f"Tipo_Incidente cargada con {len(tipo_incidente_data)} filas.")
    
    incidente_data = df[['radicado', 'clase', 'gravedad', 'diseno', 'tipo_geocod', 'OBJECTID', 'cbml', 'fecha', 'hora']].drop_duplicates(subset=['radicado'])
    incidente_data.to_sql('Incidente', conn, if_exists='append', index=False)
    print(f"Incidente cargada con {len(incidente_data)} filas.")
    
    conn.commit()
    print("Base de datos creada y cargada exitosamente.")
except Exception as e:
   print(f"Error durante creación o carga: {e}")
   conn.rollback()
   conn.close()
   raise

Tabla Ubicacion creada.
Tabla Tiempo creada.
Tabla Tipo_Incidente creada.
Tabla Incidente creada.
Ubicacion cargada con 404 filas.
Tiempo cargada con 30469 filas.
Tipo_Incidente cargada con 9 filas.
Incidente cargada con 46204 filas.
Base de datos creada y cargada exitosamente.


# 4️⃣ Evidencia con consultas SQL


In [58]:
db_file = 'incidentes_transito.db'
csv_file = 'dataset.csv'

In [59]:
#Verificar tablas existentes
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas_existentes = cursor.fetchall()
print(f"Tablas existentes en la DB: {[t[0] for t in tablas_existentes]}")

Tablas existentes en la DB: ['Ubicacion', 'Tiempo', 'Tipo_Incidente', 'Incidente']


In [60]:
#Ejecutar consultas solicitadas
tablas = ['Ubicacion', 'Tiempo', 'Tipo_Incidente', 'Incidente']

In [61]:
# Conteo de registros
print("\n=== Conteo de Registros ===")
for tabla in tablas:
    query = f"SELECT COUNT(*) FROM {tabla};"
    result = pd.read_sql_query(query, conn)
    count = result.iloc[0, 0]
    print(f"Tabla '{tabla}': {count} registros.")
    print(f"Interpretación: La tabla '{tabla}' contiene {count} filas, indicando el número total de entradas únicas. Un conteo bajo o cero sugiere problemas en la carga o datos faltantes.\n")


=== Conteo de Registros ===
Tabla 'Ubicacion': 404 registros.
Interpretación: La tabla 'Ubicacion' contiene 404 filas, indicando el número total de entradas únicas. Un conteo bajo o cero sugiere problemas en la carga o datos faltantes.

Tabla 'Tiempo': 30469 registros.
Interpretación: La tabla 'Tiempo' contiene 30469 filas, indicando el número total de entradas únicas. Un conteo bajo o cero sugiere problemas en la carga o datos faltantes.

Tabla 'Tipo_Incidente': 9 registros.
Interpretación: La tabla 'Tipo_Incidente' contiene 9 filas, indicando el número total de entradas únicas. Un conteo bajo o cero sugiere problemas en la carga o datos faltantes.

Tabla 'Incidente': 46204 registros.
Interpretación: La tabla 'Incidente' contiene 46204 filas, indicando el número total de entradas únicas. Un conteo bajo o cero sugiere problemas en la carga o datos faltantes.



In [62]:
#Nombres y tipos de columnas
print("=== Nombres y Tipos de Columnas ===")
for tabla in tablas:
    query = f"PRAGMA table_info({tabla});"
    result = pd.read_sql_query(query, conn)
    print(f"Tabla '{tabla}':")
    print(result[['name', 'type']])
    print(f"Interpretación: La tabla '{tabla}' tiene {len(result)} columnas con tipos como TEXT (cadenas), INTEGER (enteros) y REAL (flotantes), esenciales para el Modelo ER.\n")

=== Nombres y Tipos de Columnas ===
Tabla 'Ubicacion':
                name  type
0               cbml  TEXT
1          direccion  TEXT
2      direccion_enc  TEXT
3             barrio  TEXT
4             comuna  TEXT
5           longitud  REAL
6            latitud  REAL
7  x_origen_nacional  REAL
8  y_origen_nacional  REAL
9              Shape  TEXT
Interpretación: La tabla 'Ubicacion' tiene 10 columnas con tipos como TEXT (cadenas), INTEGER (enteros) y REAL (flotantes), esenciales para el Modelo ER.

Tabla 'Tiempo':
         name     type
0       fecha     TEXT
1        hora     TEXT
2         dia  INTEGER
3     periodo  INTEGER
4  dia_nombre     TEXT
5         mes  INTEGER
6  mes_nombre     TEXT
Interpretación: La tabla 'Tiempo' tiene 7 columnas con tipos como TEXT (cadenas), INTEGER (enteros) y REAL (flotantes), esenciales para el Modelo ER.

Tabla 'Tipo_Incidente':
       name  type
0     clase  TEXT
1  gravedad  TEXT
Interpretación: La tabla 'Tipo_Incidente' tiene 2 columnas con t

In [63]:
# Consulta con filtro
print("=== Consulta con Filtro ===")
query_filtro = "SELECT * FROM Incidente WHERE clase = 'Choque' LIMIT 5;"
result_filtro = pd.read_sql_query(query_filtro, conn)
print("Resultado de SELECT * FROM Incidente WHERE clase = 'Choque' LIMIT 5:")
print(result_filtro)
print(f"Interpretación: Esta consulta muestra hasta 5 incidentes de tipo 'Choque'. Si no hay resultados, no existen coincidencias; útil para análisis de patrones específicos.\n")
# Cerrar conexión
conn.close()
print("--- Fin de Consultas ---")

=== Consulta con Filtro ===
Resultado de SELECT * FROM Incidente WHERE clase = 'Choque' LIMIT 5:
  radicado   clase    gravedad        diseno  \
0  1580885  Choque      HERIDO  Tramo de via   
1  1585081  Choque  SOLO DAÑOS  Tramo de via   
2  1581508  Choque  SOLO DAÑOS  Tramo de via   
3  1578199  Choque      HERIDO  Tramo de via   
4  1591061  Choque  SOLO DAÑOS  Tramo de via   

                         tipo_geocod  OBJECTID  cbml                fecha  \
0                         Malla vial         1  1602  2017-05-05 00:00:00   
1  Malla vial aproximada: CL S 4-43A         2  1421  2017-06-06 00:00:00   
2                         Malla vial         3  1309  2017-05-10 00:00:00   
3                         Malla vial         5  0715  2017-04-14 00:00:00   
4                         Malla vial         6  0706  2017-07-25 00:00:00   

       hora  
0  02:00 PM  
1  11:20 AM  
2  12:00 PM  
3  04:30 AM  
4  10:20 AM  
Interpretación: Esta consulta muestra hasta 5 incidentes de tipo 'C